In [1]:
import os
import cv2
import numpy as np

from PIL import Image

In [68]:
def get_unpadded_borders(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    processed = cv2.morphologyEx(np.array(img), cv2.MORPH_CLOSE, kernel)
    npimg = 255 - np.array(processed)
    npimg[np.where(npimg < 16)] = 0
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    return wmin, wmax, hmin, hmax

def unpad(img):
    wmin, wmax, hmin, hmax = get_unpadded_borders(img)
    wmin = wmin - 1 if wmin > 0 else wmin
    wmax = wmax + 1 if wmax < img.width else wmax
    hmin = hmin - 1 if hmin > 0 else hmin
    hmax = hmax + 1 if hmax < img.height else hmax
    return Image.fromarray(np.array(img)[hmin:hmax,wmin:wmax])

def centering(img, new_size=128, padding_color=255):

    # Resize the image using the new dimensions
    resized_img = unpad(img)
    
    # Create a new blank image with padding
    padded_img = Image.new("L", (new_size, new_size), padding_color)
    
    # Calculate the position to paste the resized image
    left = (padded_img.width - resized_img.width) // 2
    top = (padded_img.height - resized_img.height) // 2
    right = left + resized_img.width
    bottom = top + resized_img.height
    
    # Paste the resized image onto the blank image
    padded_img.paste(resized_img, (left, top, right, bottom))
    
    return padded_img

def augmenting(img):
    img = np.array(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))

    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    processed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)
    closing = Image.fromarray(processed)
    # err = cv2.erode(processed, kernel)
    # erode = Image.fromarray(err)
    # dil = cv2.dilate(processed, kernel)
    # dilate = Image.fromarray(dil)
    return {
        "closing": closing,
        # "erode": erode,
        # "dilate":dilate,
    }

In [62]:
files = [f for f in os.listdir(fd) if f.endswith(".png") & ("플레이브밤비" not in f)]

In [71]:
for f in files:
    img = Image.open(f"{fd}/{f}")
    img = centering(img)
    img = augmenting(img)['closing']
    img.save(f"{fd}/플레이브밤비__closing__{f}")